In [5]:
# Ensure Mac ARM chip is used as GPU
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
with tf.device('/GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0])
    b = tf.constant([4.0, 5.0, 6.0])
    c = a * b
    print(c)

tf.Tensor([ 4. 10. 18.], shape=(3,), dtype=float32)


## This notebook focuses on different ways to embed text as vectors using the Keras library

In essence, the Embedding layer is like a dictionary that maps integer indices to dense vectors:

Word index -> Embedding layer -> Corresponding word vector

In [10]:
from keras.layers import Embedding
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras import preprocessing

In [9]:
# Loading the data
# Only loads the 10,000 most common words
max_features = 10000
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [13]:
model = Sequential()

# 10000 - the number of possible tokens
# 8 - the dimensionality of the encodings
model.add(Embedding(10000, 8, input_length=maxlen))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.summary()

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 161       
                                                                 
Total params: 80161 (313.13 KB)
Trainable params: 80161 (313.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 5s 7ms/step - loss: 0.6698 - acc: 0.6205 - val_loss: 0.6233 - val_acc: 0.6954
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5479 - acc: 0.7513 - val_loss: 0.5305 - val_acc: 0.7328
Epoch 3/10
625/6

The model above exhibits certain shortcomings. The biggest one is that it is unable to identify semantic connection and structure of words within a sentence. This leads to misinterpretations of natural language (i.e. "this movie is THE bomb" == "this movie is a bomb"). We will attempt better approaches using a 1-D convolutional neural network (CNN) and a recurrent neural network (RNN)